In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp

%matplotlib inline

In [2]:
# manual 2 asset check
def shrp_wts(x1,x2):
    x = np.array(x1)
    y = np.array(x2)
    
    r1 = np.mean(x)
    r2 = np.mean(y)
    var1 = np.var(x)
    var2 = np.var(y)
    c = np.cov(x,y)[0][1]
    
    w1  = (r1*var2 - r2*c) / (r1*var2 + r2*var1 - (r1+r2)*c)
    
    # limit extreme cases, prevent shorting
    #if w1<0: w1 = 0
    #if w1>1: w1 = 1
        
    w2 = 1-w1
    p = np.dot(np.column_stack((x,y)),np.array([w1,w2]))
    shrp = np.mean(p)/np.std(p)
    
    return (w1,w2,shrp)

In [3]:
n=25
#np.random.seed(23)

x = np.random.normal(.01,.01,n)
y = np.random.normal(.01,.004,n)

In [4]:
shrp_wts(x,y)

(0.14034752204987289, 0.8596524779501271, 2.410533727005176)

In [5]:
# Closed form N asset unconstrained
def calc_sharpe(m):
    '''Calculates Basic Max Sharpe for N-Asset Portfolio 
        ignores Rf rate
    
        array m must be nxm where n= returns, m = assets
        
        Constraint:
        sum(wts) = 1
    
    
        Returns tuple:
        ((wts), sharpe)
        
    '''
    cov = np.cov(m.T) #unbiased covariance, ddof=1
    u = np.mean(m,axis=0).reshape(-1,1)
    ones = np.ones_like(u)

    wtC = np.linalg.inv(cov).dot(u) / ones.T.dot(np.linalg.inv(cov)).dot(u)
    ret = u.T.dot(wtC)
    stdev = np.sqrt(wtC.T.dot(cov).dot(wtC))
    sharpe = ret/stdev


    return (wtC,sharpe)

In [6]:
calc_sharpe(np.vstack((x,y)).T)

(array([[0.14181029],
        [0.85818971]]),
 array([[2.36184627]]))

In [7]:
x = np.random.normal(.01,.01,n)
y = np.random.normal(.004,.004,n)
z = np.random.normal(.5, .001,n)

m=np.vstack([x,y,z]).T
u = np.mean(m,axis=0)

cov = np.cov(m.T)

In [8]:
calc_sharpe(m) #manual closed form

(array([[-0.00506649],
        [-0.00581975],
        [ 1.01088623]]),
 array([[457.52355386]]))

In [9]:
# Constrained N asset solution using sharpe transformation with cvxpy

y = cp.Variable(len(cov))

f =cp.quad_form(y, cov)
prob = cp.Problem(cp.Minimize(f), 
               [y@u == 1, 
                y >= 0
                
               ])
prob.solve()

x = y.value/y.value.sum()

x


array([0.01687753, 0.00718387, 0.9759386 ])

In [10]:
# Constrained N asset solution using sharpe transformation with cvxpy
# Includes min size of .1

A = np.asmatrix([[1,0,0],[0,1,0],[0,0,1]])
bounds = np.asmatrix([0.1,.1,0.1])
A_mod = A - bounds.T


y = cp.Variable(len(cov))

f =cp.quad_form(y, cov)
prob = cp.Problem(cp.Minimize(f), 
               [y@u == 1, 
                y >= 0,
                A_mod@y.T >= 0
               ])
prob.solve()

x = y.value/y.value.sum()

x

array([0.1, 0.1, 0.8])

In [11]:
# risk target example
A = np.asmatrix([[1,0,0],[0,1,0],[0,0,1]])
bounds = np.array([0.1,.1,0.1]) #change to (n,) size 

w = cp.Variable(len(cov))
ret = u.T@w 
risk = cp.quad_form(w, cov)
prob = cp.Problem(cp.Maximize(ret), 
               [cp.sum(w) == 1, 
                w >= 0,
                risk <= 2,
                A@w.T >= bounds
               ])

prob.solve()

print(w.value)
print(w.value.T.dot(cov).dot(w.value))

[0.1 0.1 0.8]
2.244021086610946e-06


In [12]:
# Max diversification portfolio

w = cp.Variable(len(cov))
#ret = u.T@w 
risk = cp.quad_form(w, cov)
prob = cp.Problem(cp.Minimize(risk), 
               [cp.sum(w) == 1, 
                w >= 0
               ])

prob.solve()

print(w.value)
print(w.value.T.dot(cov).dot(w.value))



[0.00294501 0.34190325 0.65515185]
2.4761637531643106e-06


In [ ]:
#Other cvxpy example


## OPTIMIZE PORTFOLIO WEIGHTS UNDER THE OBJECTIVE OF MAXIMIZING THE SHARPE RATIO
## WHILE CONSTRAINING THE WEIGHTS TO SECTOR BOUNDS
## PAPER: # ACCORDING TO: http://people.stat.sc.edu/sshen/events/backtesting/reference/maximizing%20the%20sharpe%20ratio.pdf

np.random.seed(101)

## NUMBER OF ASSETS
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

## GENERATE RANDOM RETURNS
return_vec = np.random.randn(n_assets, n_obs)

## SET UP PROBLEM
C = np.asmatrix(np.cov(return_vec))
mu = np.asmatrix(np.mean(return_vec,axis=1))
mu0 = np.asmatrix(-0.0075)

## INITIATE WEIGHT VARIABLE
y = cp.Variable(n_assets)

# DEFINE CONSTRAINTS AND MODIFY FOR QUADRATIC PROBLEM
A = np.asmatrix([[1,1,0,0],[-1,-1,0,0],[0,0,1,1],[0,0,-1,-1]])
bounds = np.asmatrix([0.2,-0.5,0.2,-0.8])
A_mod = A - bounds.T

## CREATE CONSTRAINTS
constraints = [(mu-mu0)@y==1,
               y >= 0,
               A_mod@y.T >= 0]

## FORM OBJECTIVE
obj = cp.Minimize(cp.quad_form(y,C))

## FORM AND SOLVE PROBLEM
prob = cp.Problem(obj, constraints)
prob.solve()

## TRANSFORM FINAL WEIGHTS
w = y.value/sum(y.value)